In [ ]:
# Inmporting libraries

import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.models import load_model

In [ ]:
# Creating model

model = Sequential();

model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", input_shape=(64,64,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=40, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=48, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten());

model.add(Dense(units=768, activation="relu"))
model.add(Dense(units=192, activation="relu"))
model.add(Dense(units=34, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
# Creating generators

train_valid_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=90, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_valid_datagen.flow_from_directory('/kaggle/input/waste-pictures/train/', target_size=(64,64))  # default batch_size(=32), class_mode(='categorical')
valid_generator = valid_test_datagen.flow_from_directory('/kaggle/input/waste-pictures/test/', target_size=(64,64))    # default batch_size(=32), class_mode(='categorical')
test_generator = valid_test_datagen.flow_from_directory('/kaggle/input/waste-classifying/', target_size=(64,64), batch_size=1, class_mode=None, shuffle=False)

In [ ]:
# Callbacks

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
# No. of batches

train_steps_size = train_generator.n // train_generator.batch_size
valid_steps_size = valid_generator.n // valid_generator.batch_size
test_steps_size = test_generator.n // test_generator.batch_size

In [ ]:
# Fitting data to model

model.fit_generator(train_generator, steps_per_epoch=train_steps_size, epochs=250, validation_data=valid_generator, validation_steps=valid_steps_size, callbacks=[es,mc])

In [ ]:
# Loading saved best model

best_model = load_model('best_model.h5')

In [ ]:
# Predicting test images

test_generator.reset()
predict = best_model.predict_generator(test_generator, steps=test_steps_size, verbose=1)

In [ ]:
# Giving labels to predicted outputs

predict_class_indices = np.argmax(predict, axis=1)

labels = dict((index, class_name) for class_name, index in train_generator.class_indices.items())
results = [labels[index] for index in predict_class_indices]

result